In [ ]:
run CELL.ipynb

In [ ]:
#these are the parameters we must specify
ionThresh = 0
openThresh = [2,15]
numCells = 7
ionChans = [12,1]

#these are the parameters that have preset values
#can still change if desired
#C = [25, 150, 25]
#G = [0.133,0.015,0.133]
#N = [2,12,2]
#gapJunction = 1
#theFilter = 0.01
#NiGAP = 80
#NiMARG = 4200
#pokeHoleMARG = False

#we want to vary gapJunction, so we'll substitute in the values
#in the list below
gJs = [0.0001,0.001,0.01,0.1,1]

rateMultiplier = {'0.2':37.5,'0.01':15000}

rates = []

for gJ in gJs:
    myModel = createModel(numCells, ionThresh, openThresh, ionChans, gapJunct = gJ)
    depol = runModel(myModel)

    fitcoeffs = np.round(np.polyfit(range(2,len(depol)), depol[1:-1], 1),3)
    thePoly = np.poly1d(fitcoeffs)
    theRate =  1/fitcoeffs[0]

    rates.append(theRate*rateMultiplier[str(theFilter)])

gJs = [0.1,1]
rates = [2,4]
figl = plt.figure()
axl = figl.add_axes([0,0,1,1])
axl.set_xlabel('This is the x-axis (Gap Junction Coefficients')
axl.set_ylabel('This is the y-axis (Conduction Velocity (cm/s))')
axl.set_title('This is the Title')
axl.scatter(gJs, rates, label = 'Can Label Like This')
axl.plot(gJs, rates)
axl.legend()
figl.savefig('GraphTitle.png',bbox_inches='tight')